# Step 3 - K-Modes Clustering Analysis

This code requires the following file to run:
- St2_person_level_engineered.parquet

-------------------------

This is the code for the third step of our pipeline.
It is divided into two main parts, Clustering and Cluster Visualization. In summary:

*For Clustering*
* Filter dataset to drivers only
* Select relevant categorical and binary features for clustering
* Apply K-Modes clustering algorithm for categorical data
* Determine optimal number of clusters using elbow method
* Evaluate clustering quality using silhouette score and homogeneity metrics
* Save cluster assignments for subsequent analysis

*For Cluster Visualization*
* Visualize cluster characteristics with feature prevalence histograms

K-Modes clustering was chosen because it is specifically designed for categorical data, unlike K-Means which only works with numerical data. This algorithm identifies modes (most common values) instead of means, making it ideal for our FARS dataset which contains mostly categorical variables.

A limitation of this approach is that the optimal k selection using the elbow method can be subjective (in this code the optimal k does not coincide with the select k for clustering).

### How to run the code:

1) Run libraries
2) Run all the sections in order (top to bottom)
3) Run the Use section
4) Optional: review the code of each section

In [1]:
import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

### Step 3.1: Data Loading and Filtering

Load engineered features and filter to drivers only

In [2]:
def load_and_filter_drivers(input_file: Path) -> pd.DataFrame:

    # Load data
    df = pd.read_parquet(input_file)
    print(f"   Total persons in dataset: {len(df):,}")
    
    # Filter to drivers only (PER_TYP == 1)
    df_drivers = df[df['PER_TYP'] == 1].copy()
    print(f"   Total drivers: {len(df_drivers):,}")
    
    return df_drivers

### Step 3.2: Feature Selection and Preparation

Select relevant features for clustering and handle missing values

In [3]:
def prepare_clustering_features(df: pd.DataFrame, features_config: dict) -> tuple:

    # Extract feature configuration (Review USE to modify selected features)
    binary_features = features_config.get('binary', [])
    categorical_features = features_config.get('categorical', [])
    
    # Combine all features
    all_features = binary_features + categorical_features
    
    print(f"   Using {len(all_features)} features for clustering:")
    print(f"      - Binary features: {len(binary_features)}")
    print(f"      - Categorical features: {len(categorical_features)} ({', '.join(categorical_features)})")
    
    # Select features
    df_cluster = df[all_features].copy()
    
    # Handle missing values - drop rows with any missing values
    rows_before = len(df_cluster)
    df_cluster = df_cluster.dropna()
    rows_after = len(df_cluster)
    rows_removed = rows_before - rows_after
    
    if rows_removed > 0:
        print(f"\n   Removed {rows_removed:,} rows with missing values ({rows_removed/rows_before*100:.1f}%)")
    print(f"   Remaining drivers: {rows_after:,}")
    
    return df_cluster, all_features

### Step 3.3: Elbow Method for Optimal K

Determine optimal number of clusters using elbow method

In [4]:
def find_optimal_k(X: np.ndarray, k_range: range = range(2, 11)) -> tuple:

    # In USE the range of k values can be changed
    print(f"   Testing k values from {min(k_range)} to {max(k_range)}...")
    
    costs = {}
    
    for k in k_range:
        km = KModes(n_clusters=k, init='Cao', n_init=5, verbose=0, random_state=42)
        km.fit(X)
        costs[k] = km.cost_
        print(f"      k={k}, cost={km.cost_:.2f}")
    
    # Simple elbow detection: find the point with maximum decrease
    cost_list = list(costs.values())
    k_list = list(costs.keys())
    
    # Calculate rate of change
    rate_changes = []
    for i in range(1, len(cost_list)):
        rate_changes.append(cost_list[i-1] - cost_list[i])
    
    # Find elbow (where rate of decrease slows down most)
    if len(rate_changes) >= 2:
        second_derivatives = []
        for i in range(1, len(rate_changes)):
            second_derivatives.append(rate_changes[i-1] - rate_changes[i])
        optimal_k = k_list[second_derivatives.index(max(second_derivatives)) + 2]
    else:
        optimal_k = k_list[1]  # Default to k=3 if calculation fails
    
    print(f"\n   Optimal k based on elbow method: {optimal_k}")
    
    return optimal_k, costs

### Step 3.4: K-Modes Clustering

*IMPORTANT NOTE:*

In this case, the number of clusters was hardcoded (k=6) as it showed similar results and made more sense within the followed context.
The code includes the possibility to run the clustering with the automatically detected optimal k.

In [5]:
def perform_kmodes_clustering(X: np.ndarray, n_clusters: int, verbose: int = 1) -> KModes:

    print(f"\n   Running final K-Modes clustering with k={n_clusters}...")
    
    km = KModes(n_clusters=6, init='Cao', n_init=10, verbose=1, random_state=42) #Hardcoded version
    # Automatic version below:
    # km = KModes(n_clusters=n_clusters, init='Huang', n_init=1, verbose=verbose)
    km.fit(X)
    
    print(f"\n   Clustering complete!")
    print(f"   Final cost: {km.cost_:.2f}")
    
    return km

### Step 3.5: Clustering Evaluation

Calculate clustering quality metrics (Silhouette Score)

In [6]:
def evaluate_clustering(X: np.ndarray, labels: np.ndarray, sample_size: int = 10000) -> dict:

    # Convert categorical to numeric for silhouette calculation
    X_numeric = np.zeros(X.shape)
    for i in range(X.shape[1]):
        unique_vals = np.unique(X[:, i])
        val_to_num = {val: idx for idx, val in enumerate(unique_vals)}
        X_numeric[:, i] = [val_to_num[val] for val in X[:, i]]
    
    # Calculate silhouette score on a sample (for efficiency)
    if len(X_numeric) > sample_size:  #Sample size established as 10000
        print(f"   Calculating Silhouette Score (using sample of {sample_size:,} drivers)...")
        sample_idx = np.random.choice(len(X_numeric), sample_size, replace=False)
        X_sample = X_numeric[sample_idx]
        labels_sample = labels[sample_idx]
        silhouette = silhouette_score(X_sample, labels_sample)
    else:
        print("   Calculating Silhouette Score...")
        silhouette = silhouette_score(X_numeric, labels)
    
    print(f"      Silhouette Score: {silhouette:.4f}")
    
    return {'silhouette_score': silhouette}

### Step 3.6: Results Saving and Summary

Save cluster assignments and generate summary statistics

In [7]:
def save_clustering_results(df_original: pd.DataFrame, df_cluster: pd.DataFrame, 
                           labels: np.ndarray, output_file: Path, 
                           costs: dict = None) -> pd.DataFrame:
   
    # Add cluster labels to the filtered dataframe
    df_cluster_result = df_cluster.copy()
    df_cluster_result['cluster'] = labels
    
    # Save results
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    df_cluster_result.to_parquet(output_path)
    print(f"   Cluster assignments saved to: {output_path}")
    
    # Print cluster distribution
    print("\n   Cluster size distribution:")
    cluster_counts = pd.Series(labels).value_counts().sort_index()
    for cluster_id, count in cluster_counts.items():
        pct = count / len(labels) * 100
        print(f"      Cluster {cluster_id}: {count:,} drivers ({pct:.1f}%)")
    
    # Calculate within-cluster homogeneity
    print("\n   Computing within-cluster homogeneity...")
    X_array = df_cluster.values
    
    for cluster_id in sorted(np.unique(labels)):
        cluster_mask = labels == cluster_id
        cluster_data = X_array[cluster_mask]
        
        # Calculate mode agreement for each feature
        agreements = []
        for col_idx in range(cluster_data.shape[1]):
            col_data = cluster_data[:, col_idx]
            mode_val = pd.Series(col_data).mode()[0]
            agreement = (col_data == mode_val).sum() / len(col_data)
            agreements.append(agreement)
        
        avg_homogeneity = np.mean(agreements) * 100
        print(f"      Cluster {cluster_id}: {avg_homogeneity:.1f}% average feature agreement")
    
    overall_homogeneity = np.mean([np.mean([(X_array[labels == c, col] == 
                                             pd.Series(X_array[labels == c, col]).mode()[0]).sum() / 
                                            (labels == c).sum() 
                                            for col in range(X_array.shape[1])]) 
                                  for c in np.unique(labels)]) * 100
    print(f"\n   Overall average homogeneity: {overall_homogeneity:.1f}%")
    
    # Save elbow plot if costs provided
    if costs is not None:
        plt.figure(figsize=(10, 6))
        k_values = list(costs.keys())
        cost_values = list(costs.values())
        
        plt.plot(k_values, cost_values, 'bo-', linewidth=2, markersize=8)
        plt.xlabel('Number of Clusters (k)', fontsize=12, fontweight='bold')
        plt.ylabel('Cost', fontsize=12, fontweight='bold')
        plt.title('K-Modes Elbow Method', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        output_path = Path(output_file)

        elbow_plot_path = Path("Results") / "Optimal_K" / "kmodes_elbow.png"
        plt.savefig(elbow_plot_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"\n   Elbow plot saved to: {elbow_plot_path}")
    
    return df_cluster_result

### Step 3.7: Cluster Visualization

Create visualizations for cluster characteristics

In [8]:
def create_binary_features(df: pd.DataFrame, cluster_id: int, 
                          binary_cols: list, categorical_cols: list) -> dict:

    cluster_data = df[df['cluster'] == cluster_id]
    features = {}
    
    # Binary features (show both 0 and 1)
    for col in binary_cols:
        if col in cluster_data.columns:
            features[f"{col}=1"] = (cluster_data[col] == 1).sum() / len(cluster_data) * 100
            features[f"{col}=0"] = (cluster_data[col] == 0).sum() / len(cluster_data) * 100
    
    # Categorical features (show all categories)
    for col in categorical_cols:
        if col in cluster_data.columns:
            value_counts = cluster_data[col].value_counts(normalize=True, dropna=False) * 100
            for val, pct in value_counts.items():
                features[f"{col}={val}"] = pct
    
    return features


def visualize_clusters(df_clusters: pd.DataFrame, binary_features: list, 
                      categorical_features: list, output_dir: Path, 
                      prevalence_threshold: float = 65.0):
 
    clusters = sorted(df_clusters['cluster'].unique())
    n_clusters = len(clusters)
    print(f"   Number of clusters: {n_clusters}")
    
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    for cluster_id in clusters:
        # Get feature prevalence for this cluster
        features_dict = create_binary_features(df_clusters, cluster_id, 
                                               binary_features, categorical_features)
        
        # Filter to high prevalence features
        high_prev_features = {k: v for k, v in features_dict.items() 
                             if v > prevalence_threshold}
        
        # Sort by prevalence (descending)
        sorted_features = dict(sorted(high_prev_features.items(), 
                                     key=lambda x: x[1], reverse=True))
        
        # Get cluster size
        cluster_size = len(df_clusters[df_clusters['cluster'] == cluster_id])
        
        # Create figure
        fig, ax = plt.subplots(figsize=(14, max(6, len(sorted_features) * 0.4)))
        
        if len(sorted_features) > 0:
            names = list(sorted_features.keys())
            values = list(sorted_features.values())
            
            # Create horizontal bar chart
            bars = ax.barh(names, values, color='steelblue', 
                          edgecolor='navy', linewidth=0.5)
            
            # Styling
            ax.set_xlabel('Prevalence (%)', fontsize=12, fontweight='bold')
            ax.set_title(
                f'Cluster {cluster_id}: Features >{prevalence_threshold}% Prevalent '
                f'(n={cluster_size:,} drivers)',
                fontsize=14, fontweight='bold', pad=15
            )
            ax.axvline(x=prevalence_threshold, color='red', linestyle='--', 
                      linewidth=2, label=f'{prevalence_threshold}% threshold')
            ax.set_xlim(prevalence_threshold-5, 105)
            ax.legend(loc='lower right', fontsize=10)
            ax.grid(axis='x', alpha=0.3)
            
            # Add value labels on bars
            for bar, val in zip(bars, values):
                ax.text(val + 0.5, bar.get_y() + bar.get_height()/2,
                       f'{val:.1f}%', va='center', ha='left', 
                       fontsize=9, fontweight='bold')
            
            ax.tick_params(axis='y', labelsize=10)
            ax.tick_params(axis='x', labelsize=10)
        else:
            # No high-prevalence features
            ax.text(0.5, 0.5, 
                   f'No features >{prevalence_threshold}% prevalent\n'
                   f'(Cluster size: {cluster_size:,} drivers)',
                   ha='center', va='center', fontsize=13, fontweight='bold',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.axis('off')
        
        plt.tight_layout()
        
        # Save visualization
        output_file = Path("Results") / "Individual_Cluster_Results" / f"cluster_{cluster_id}_features.png"
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"   Cluster {cluster_id}: {len(sorted_features)} features "
              f">{prevalence_threshold}% prevalent")
        plt.close()
    
    print(f"\n   Visualizations saved to: {output_dir}")

### Pipeline

Run the complete clustering analysis

In [9]:
def run_clustering_pipeline(
    input_file: Path,
    output_file: Path,
    features_config: dict,
    k_range: range = range(2, 11),
    visualize: bool = True,
    prevalence_threshold: float = 65.0
) -> pd.DataFrame:

    # Step 1: Load and filter to drivers
    df_drivers = load_and_filter_drivers(input_file)
    
    # Step 2: Prepare features
    df_cluster, feature_names = prepare_clustering_features(df_drivers, features_config)
    
    # Step 3: Prepare data summary

    # Get accident-level statistics
    if 'ST_CASE' in df_drivers.columns:
        unique_accidents = df_cluster.index.map(
            lambda idx: df_drivers.loc[idx, 'ST_CASE'] 
            if idx in df_drivers.index else None
        ).nunique()
        print(f"   Total drivers for clustering: {len(df_cluster):,}")
        print(f"   Unique accidents represented: {unique_accidents:,}")
        print(f"   Average drivers per accident: {len(df_cluster)/unique_accidents:.2f}")
    print(f"   Final clustering dataset shape: {df_cluster.shape}")
    
    # Convert to numpy array
    X = df_cluster.values
    
    # Step 4: Find optimal k
    optimal_k, costs = find_optimal_k(X, k_range)
    
    # Step 5: Perform clustering
    km_model = perform_kmodes_clustering(X, optimal_k, verbose=1)
    
    # Step 6: Evaluate
    metrics = evaluate_clustering(X, km_model.labels_)
    
    # Step 7: Save results
    df_results = save_clustering_results(
        df_drivers, df_cluster, km_model.labels_, output_file, costs
    )
    
    # Step 8: Visualize (optional)
    if visualize:
        visualize_clusters(
            df_results, 
            features_config['binary'],
            features_config['categorical'],
            Path(output_file),
            prevalence_threshold
        )
    
    # Final summary
    print(f"\nSummary:")
    print(f"   • Total drivers clustered: {len(df_cluster):,}")
    print(f"   • Number of clusters: {optimal_k}")
    print(f"   • Silhouette score: {metrics['silhouette_score']:.4f}")
    print(f"   • Features used: {len(feature_names)}")
    print(f"\nOutput files:")
    print(f"   • {output_file}")
    if visualize:
        print(f"   • Results")
    
    return df_results

### USE

Configure INPUT and OUTPUT paths and feature selection

In [10]:
# INPUT/OUTPUT Configuration
INPUT_FILE = Path("Dataset/St2_person_level_engineered.parquet")
OUTPUT_FILE = Path("Dataset/St3_fatal_accident_clusters.parquet")

# Features Configuration (the ones used)
FEATURES_CONFIG = {
    'binary': [
        'RUSH_HOUR',
        'MALE',
        'ADVERSE_WEATHER',
        'DARK_CONDITIONS',
        'OLD_VEHICLE',
        'PASSENGER_CAR',
        'LARGE_TRUCK',
        'MOTORCYCLE',
        'URBAN',
        'INTERSTATE',
        'INTERSECTION',
        'WORK_ZONE_CRASH',
        'ROLLOVER_CRASH',
        'FIRE',
        'WEEKEND_FLAG'
    ],
    'categorical': [
        'TIME_OF_DAY',
        'SEASON',
        'AGE_GROUP'
    ]
}

# Clustering Parameters
K_RANGE = range(2, 11)  # Test k from 2 to 10
VISUALIZE = True  # Create cluster visualizations
PREVALENCE_THRESHOLD = 65.0  # Show features above this % in visualizations

# RUN PIPELINE

if not INPUT_FILE.exists():
    print(f"\nInput file not found: {INPUT_FILE}")
    print("Run Step 2 (Feature Engineering) first")
else:
    df_clustered = run_clustering_pipeline(
        input_file=INPUT_FILE,
        output_file=OUTPUT_FILE,
        features_config=FEATURES_CONFIG,
        k_range=K_RANGE,
        visualize=VISUALIZE,
        prevalence_threshold=PREVALENCE_THRESHOLD
    )

   Total persons in dataset: 92,400
   Total drivers: 57,939
   Using 18 features for clustering:
      - Binary features: 15
      - Categorical features: 3 (TIME_OF_DAY, SEASON, AGE_GROUP)

   Removed 2,214 rows with missing values (3.8%)
   Remaining drivers: 55,725
   Total drivers for clustering: 55,725
   Unique accidents represented: 35,995
   Average drivers per accident: 1.55
   Final clustering dataset shape: (55725, 18)
   Testing k values from 2 to 10...
      k=2, cost=269056.00
      k=3, cost=256796.00
      k=4, cost=242167.00
      k=5, cost=234373.00
      k=6, cost=227144.00
      k=7, cost=227008.00
      k=8, cost=221670.00
      k=9, cost=214055.00
      k=10, cost=207694.00

   Optimal k based on elbow method: 7

   Running final K-Modes clustering with k=7...
Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 14829, cost: 22922